In [33]:
import pandas as pd
import numpy as np 
from ast import literal_eval
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD

import warnings; warnings.simplefilter('ignore')

In [2]:
md = pd.read_csv('./movie_dataset/movies_metadata.csv')
links_small = pd.read_csv('./movie_dataset/links_small.csv')

/tmp/ipykernel_8181/1058398154.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv('./movie_dataset/movies_metadata.csv')


In [3]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [4]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [19]:
def weighted_rating(x, quantile):
    C = x[x['vote_average'].notnull()]['vote_average'].astype('int').mean()
    vote_counts = x[x['vote_count'].notnull()]['vote_count'].astype('int')
    m = vote_counts.quantile(quantile)
    data = x[(x['vote_count'] >= m) & (x['vote_count'].notnull()) & (x['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    data['vote_count'] = data['vote_count'].astype('int')
    data['vote_average'] = data['vote_average'].astype('int')
    v = data['vote_count']
    R = data['vote_average']
    data['wr'] = (v/(v+m) * R) + (m/(m+v) * C)
    return data

In [6]:
data = weighted_rating(md, 0.95)

In [7]:
data = data.sort_values('wr', ascending=False).head(250)

In [9]:
data.head()

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787


In [13]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [22]:
romance = gen_md[gen_md['genre'] == 'Romance']
romance_wr = weighted_rating(romance, 0.85)
romance_wr = romance_wr.sort_values('wr', ascending=False)
romance_wr

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
...,...,...,...,...,...,...
3698,Nutty Professor II: The Klumps,2000,338,4,6.671316,4.297594
252,Junior,1994,381,4,6.710215,4.270477
4240,Dr. Dolittle 2,2001,422,4,7.906683,4.248855
10135,Bewitched,2005,463,4,9.276085,4.230435


In [23]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')

In [24]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

In [28]:
sm = smd.copy()
sm['tagline'] = smd['tagline'].fillna('')
sm['description'] = smd['overview'] + smd['tagline']
sm['description'] = smd['description'].fillna('')

In [37]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(sm['description'])

In [38]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [39]:
sm = sm.reset_index()
titles = smd['title']
indices = pd.Series(sm.index, index=sm['title'])

In [40]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [41]:
get_recommendations('The Godfather').head(10)

1178      The Godfather: Part II
21614                 The Family
4324                        Made
5433          Johnny Dangerously
29                Shanghai Triad
8364                        Fury
2891              American Movie
1914     The Godfather: Part III
5476                     8 Women
2587               Summer of Sam
Name: title, dtype: object

In [42]:
credits = pd.read_csv('./movie_dataset/credits.csv')
keywords = pd.read_csv('./movie_dataset/keywords.csv')

In [43]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [44]:
md.shape

(45463, 25)

In [45]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [46]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

In [47]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [48]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [49]:
smd['director'] = smd['crew'].apply(get_director)
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [50]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [51]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [52]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [53]:
s = s.value_counts()
s[:5]

keyword
independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: count, dtype: int64

In [58]:
s = s[s > 1]

In [59]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [60]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [61]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [62]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [63]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [64]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [65]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

In [76]:
ratings = pd.read_csv('./movie_dataset/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
